<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/11-Flask/G-Using_Forms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Forms

Now let's see how we can use HTML forms to submit our own data.

HTML forms allow us to get user entries and then use the submitted values as parameters in our code. (See [W3 Schools](https://www.w3schools.com/html/html_forms.asp) for more details.)

Suppose that you want to have the ability to search the Citibike stations by their name, and get back a list of the stations that contain the search string. A simple HTML form that can do that is listed below. Save the file  `search_stations.html` in your `templates` folder, and add a new route in your webserver.py, so that you can get back this page, when you visit the main page of your website.


#### `search_stations.html`

```html
<!DOCTYPE html>
<html>

<head>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css"
        integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous">
    <title>Search Citibike Stations</title>
</head>

<body>
    <div class="container">
        <div class="panel panel-info">
            <div class="panel-heading" align="center">
                <h1>
                    <form action="/search">
                        Station Name<br>
                        <input type="text" name="name"><br>
                        <input class="btn btn-primary" type="submit" value="Submit">
          </form>
                </h1>
            </div>
        </div>
    </div>
</body>
```

#### `webserver.py`

```python
@app.route('/')
def home():
    return render_template('search_stations.html')
```

A key part of the form is the `action` script.

This defines what is the URL that we should go to, when we submit the form. In our case, it will go to the `http://<your IP>:5000/search` with the parameter `name` having the value of the text that the user entered.

For example, if we enter `Bleecker` on the form, the form will direct us to the URL `http://<your IP>:5000/search?name=Bleecker`.

Now, we need to specify the corresponding route in our webserver (i.e., the `/search` route) and define what it will do.

But first, we will allow our API to receive a parameter first, and allow to search the stations by name:

In [ ]:
@app.route('/citibike_api')
def citibike_stations():

  search_query = request.args.get('name')

  if search_query is None:
    sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
  else:
    sql = '''SELECT DISTINCT id AS station_id, name, capacity, lat, lon
                FROM status_fall2017
                WHERE name LIKE %(station_name_query)s
          '''

  with engine.connect() as con:
    stations = pd.read_sql(sql, con=con, params={"station_name_query": search_query})

  list_of_stations = stations.to_dict(orient='records')

  api_results = {"stations": list_of_stations}

  # We JSON-ify our dictionary and return it as the API response
  return jsonify(api_results)